# Sujood and Guillem, 2 methods:

https://scikit-learn.org/1.5/modules/classification_threshold.html    -> LOOK at it for part 1 and part 2.

https://scikit-learn.org/1.5/modules/feature_selection.html
    
Separate into training and testing, as we will we doing some hyperparameter tuning.
- For now we don't balance the original data, we may do it inside the model.
- For now we are not standarizing anythin either, this can be changed and optimized.
- Drop variables in full dataset we have that are full zeros or full ones we can never gain any information from those.

Approach from class:

1. BASELINE: Lasso for different alphas, this will give us different features, and data coded -1,1 so we can interpret as a linear probability model. We do not change the threshold so the threshold is 0. Advantage, no hypertuning, we can do this on the whole data. Alpha range from: {0.001, 0.040 tentative} and we get balanced accuracy on the test of 0.8 to below. 
  2. (1.1) We start tuning the threshold. This means hyper parameter tuning. We use 5 fold cross validation. The cross validation only occurs within the training data. What are we hyperparameter tuning? We just change the threshold, so we hyperparameter tune over optimal threshold and we take the average thresholds over the folds as the optimal threshold. Then we fit a lasso on the whole training data and we get a prediction based on optimal threshold for maximizing balanced accuracy. We iterate this procedure across many alphas. Gives like 0.87 for 20 features more or less.
     
Both approaches give us a plot of features vs balanced accuracy. We can consider them different models or we can consider them one. 

3. Forward feature selection:
- We need to cut the initial number of variables somehow or this gets too computationally expensive.
- Lasso is an aggressive way of cutting the variables, I think it will select p_lasso = N as a maximum, so it is very greedy. Use 0.0001 687 features.
- Then this forward feature selection is another greedy approach. But nothing wrong with being greedy.
Forward feature selection, does badly with local minima, maybe we have to tune that. Auto stops when two consecutive iterations can't improve. So in a sense prevents from overfitting but it also prevents from seeing beyond the slope.


Cooking our own approaches: 

1. Lasso + SVM ():
   For different alphas we train a Lasso on the whole data. We get a subset of X; based on which coefficients for the Lasso are not equal to 0. We then trained an out of the box SVC(radial kernel, we do not need any modifications, it just uses predict.). 
   WE DO SET: Class weights to balanced, this improves performance a bit, careful about doing this ad-hoc hyper parameter tuning. This was my hallucinations or not? WE should be careful.
    7 features with 0.91 accuracy.
    
2. Lasso + SVM() + threshold tuning ():
For different alphas we train a Lasso on the whole data. We get a subset of X; based on which coefficients for the Lasso are not equal to 0. We then trained an out of the box SVC(radial kernel, probability=True). This returns a probability for each class, we use the probability for the positive class to tune the threshold.
    0.89 without class balance or anything.





USE THIS THRESHOLD TUNING FOR PART 1, let's see if I can get a bit more.    

Why is variance thresholding used? It doesn't make too much sense to me.

In [100]:
import os
import pandas as pd
import datetime
import time
import numpy as np
import json # Export dictionary of tuned parameters.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold # For hyperparameter tuning.


# Evaluation metrics import
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, 
    confusion_matrix
)

# Models import
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# Visualisations import
import seaborn as sns
import matplotlib.pyplot as plt

# Baseline approach, Lasso:

The label is coded as -1 and 1 we can use Lasso as a classifier.

We can try many lambdas and as a number of features we can get the ones with non zero beta coefficients.

We fit on the training set and then we get

- Regularize the data, use standard scaler.


# TIPS:
FEATURE SELECTION:

Feature selection methods, LASSO, and other lp penalties, LASSO IS GREAT because it sets to 0. Others I haven't tested.

FORGET ABOUT AIC, BIC, subset selection, we can't fit those models it takes too long.

Play around Lasso lambda to get more or less features.

BE GREEDY! lASSO on subsets of the data. keep more variables as well. Low time approaches.

https://stats.stackexchange.com/questions/27300/using-principal-component-analysis-pca-for-feature-selection (idk if this says something useful might be interesting to check.) And read the p>>n chapter of book for other methods. 

<hr>
CLASSIFICATION:

DICHOTOMY: CLASSIFICATION AND FEATURE SELECTION TO BE TREATED LIKE ORTHOGONAL ELEMENTS.

Get variables from Lasso non zero coefficients and then fit a model, go big, svm, rf, whatever, don't try to classify using a Lasso. 

<hr>
WORKFLOW:

If we don't hyper parameter tune there is no reason we can't use CV for an estimate of the test error. We have few datapoints, every point is worth saving. ONLY IF WE DON'T HYPER PARAM TUNE WE CAN DO THIS, but baseline classification algorithms are already very good!.

CLASSES VERY UNBALANCED, WE NEED TO TAKE THIS INTO ACCOUNT:
REWEIGHTING things might be good for this. Some algorithms have options to reweight like rf or whatever.



In [3]:
#This is for data, which is most important split.
RANDOM_STATE = 42

In [4]:
def load_data(file_path):
    return pd.read_csv(file_path, compression='gzip')


In [5]:
file_path = "Datasets/data2.csv.gz" 

# Load the dataset
data = load_data(file_path)

In [6]:
data

,label,0,1,2,3,4,5,6,7,8,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
0,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
796,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
def thresholding(threshold, y):
    # Ensure the input is a NumPy array
    y = np.array(y, copy=True)
    
    # Apply the thresholding logic
    y_binarized = np.where(y >= threshold, 1, 0)
    
    return y_binarized



In [108]:
kf = KFold(n_splits=5)
thresholds = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9]
fold_optimal = []
for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    x_train_cv = X_train.iloc[train_index, :]
    x_test_cv = X_train.iloc[test_index,:]
    y_train_cv = y_train.iloc[train_index]
    y_test_cv = y_train.iloc[test_index]
    
    # Fit the model
    lasso = Lasso(alpha= 0.1)
    lasso.fit(x_train_cv, y_train_cv)
    y_pred = lasso.predict(x_test_cv)

    fold_scores = []
    for threshold in thresholds:
        
        y_label = thresholding(threshold, y_pred)
        fold_scores.append(balanced_accuracy_score(y_test_cv, y_label))

    fold_optimal.append(thresholds[np.argmax(fold_scores)])

/opt/anaconda3/envs/m_learn/lib/python3.13/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/anaconda3/envs/m_learn/lib/python3.13/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/anaconda3/envs/m_learn/lib/python3.13/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/anaconda3/envs/m_learn/lib/python3.13/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/anaconda3/envs/m_learn/lib/python3.13/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred con

In [7]:
data["label"].value_counts()
# DATA IS VERY UNBALANCED, SOME ESTIMATORS HAVE OPTIONS TO REWEIGHT THE DATA!!

label
-1    722
 1     78
Name: count, dtype: int64

In [55]:

balanced_accuracy = balanced_accuracy_score(y_TEST_G, y_pred)


In [56]:
# This gets worse the bigger the test, meaning we train with less data. Try pure CV and reweighting methods.
balanced_accuracy
# 0.9 balanced accuracy as baseline if I am not mistaken?

np.float64(0.7743055555555556)

In [95]:
data["label"].iloc[[1,2,3]]

1   -1
2   -1
3   -1
Name: label, dtype: int64

In [65]:
for i in range(1,1000,5):
    tic = time.time()
    lasso = Lasso(alpha=i/10000)
    
    lasso.fit(X_TRAIN_G,y_TRAIN_G)
    toc = time.time()
    rf = RandomForestClassifier(class_weight="balanced")
    rf.fit(X_TRAIN_G[:,lasso.coef_!=0],y_TRAIN_G)
    y_pred = rf.predict(X_TEST_G[:,lasso.coef_!=0])
    balanced_accuracy = balanced_accuracy_score(y_TEST_G, y_pred)

    
    print(f"""
    Lasso alpha{i/10000}, number of parameters for the random forest {X_TRAIN_G[:,lasso.coef_!=0].shape[1]}, 
    balanced accuracy {balanced_accuracy}, lasso fit time {toc-tic}
    """
         )

/opt/anaconda3/envs/m_learn/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.531e-03, tolerance: 5.599e-03
  model = cd_fast.enet_coordinate_descent(



    Lasso alpha0.0001, number of parameters for the random forest 638, 
    balanced accuracy 0.7743055555555556, lasso fit time 14.35176134109497
    

    Lasso alpha0.0006, number of parameters for the random forest 491, 
    balanced accuracy 0.7743055555555556, lasso fit time 5.116127014160156
    

    Lasso alpha0.0011, number of parameters for the random forest 374, 
    balanced accuracy 0.8055555555555556, lasso fit time 2.6300270557403564
    

    Lasso alpha0.0016, number of parameters for the random forest 294, 
    balanced accuracy 0.8090277777777778, lasso fit time 1.9154973030090332
    

    Lasso alpha0.0021, number of parameters for the random forest 226, 
    balanced accuracy 0.8055555555555556, lasso fit time 1.4705348014831543
    

    Lasso alpha0.0026, number of parameters for the random forest 169, 
    balanced accuracy 0.8055555555555556, lasso fit time 1.3455009460449219
    

    Lasso alpha0.0031, number of parameters for the random forest 135, 
    b

ValueError: Found array with 0 feature(s) (shape=(640, 0)) while a minimum of 1 is required by RandomForestClassifier.